In [2]:
root = "/kaggle/input/d/lawliet07/rnd-beg-dataset/test_images/"

In [3]:
import os
len(os.listdir(root))

45

In [4]:
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
import torchvision.transforms as transforms
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

In [24]:
# transform = torchvision.transforms.Compose([
#     transforms.ToTensor()
# ])


In [5]:
dataset_paths = os.listdir(root)
dataset_paths = [root + img for img in dataset_paths]
dataset_paths[:5]

['/kaggle/input/d/lawliet07/rnd-beg-dataset/test_images/test_buddha2.jpg',
 '/kaggle/input/d/lawliet07/rnd-beg-dataset/test_images/test_buddha27.jpg',
 '/kaggle/input/d/lawliet07/rnd-beg-dataset/test_images/test_buddha41.jpg',
 '/kaggle/input/d/lawliet07/rnd-beg-dataset/test_images/test_buddha45.jpg',
 '/kaggle/input/d/lawliet07/rnd-beg-dataset/test_images/test_buddha11.jpg']

In [6]:
label_map = {"broken" : 0,
             "buddha" : 1}

In [7]:
import xml.etree.ElementTree as ET
annotation_path = "/kaggle/input/buddha-annotations-coco/"
anno_data = os.listdir(annotation_path)
anno_data = [annotation_path + anno for anno in anno_data]
boxes = []
labels = []
for annotation in anno_data:
    tree = ET.parse(annotation)
    root = tree.getroot()
    all_box = []
    all_label = []
    for obj in root.findall("object"):
            bbox = obj.find("bndbox")
            xmin = int(bbox.find("xmin").text)
            ymin = int(bbox.find("ymin").text)
            xmax = int(bbox.find("xmax").text)
            ymax = int(bbox.find("ymax").text)
            all_box.append([xmin, ymin, xmax, ymax])
            label = obj.find("name").text
            all_label.append(label_map[label])
    boxes.append(torch.as_tensor(all_box))
    labels.append(torch.as_tensor(all_label))
    

In [28]:
# labels[:5]

In [8]:
target = []
# boxes = torch.as_tensor(boxes)
# labels = torch.as_tensor(labels)
for i in range(len(boxes)):
    target.append({"boxes": boxes[i], "labels": labels[i]})

In [9]:
boxes[0]

tensor([[ 71,  25, 110,  94],
        [106,   0, 159,  27],
        [152,  29, 187, 101],
        [ 23,   0, 243, 152]])

In [10]:
target[0]

{'boxes': tensor([[ 71,  25, 110,  94],
         [106,   0, 159,  27],
         [152,  29, 187, 101],
         [ 23,   0, 243, 152]]),
 'labels': tensor([0, 0, 0, 1])}

In [11]:
dataset = []
for img_path in dataset_paths:
    img = Image.open(img_path).convert("RGB")
    img = transforms.Resize((512, 512))(img)
    img = transforms.ToTensor()(img)
    dataset.append(img)

In [12]:
# len(dataset[0])
(dataset[0]).shape

torch.Size([3, 512, 512])

In [18]:
for i in range(len(dataset)):
    dataset[i] = [dataset[i], target[i]]
# dataset[:5]

In [13]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [14]:
data_loader = DataLoader(
    dataset,
    batch_size=2,  # You can adjust this as needed
    shuffle=True,
    collate_fn=collate_fn
)

In [15]:
# Define Faster R-CNN model
backbone = torchvision.models.resnet50(pretrained=True)
backbone.out_channels = 2048  # Set output channels for ResNet backbone
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                 output_size=7,
                                                 sampling_ratio=2)
model = FasterRCNN(backbone,
                   num_classes=3,  # Adjust based on your dataset's number of classes
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)

# Define optimizer and learning rate scheduler
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Training loop (simplified)
# model.train()
# num_epochs = 10
# for epoch in range(num_epochs):
#     for images, targets in data_loader:
#         optimizer.zero_grad()
#         loss_dict = model(images, targets)
#         losses = sum(loss for loss in loss_dict.values())
#         losses.backward()
#         optimizer.step()

#     lr_scheduler.step()
#     print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {losses.item()}")

# # Save the trained model
# torch.save(model.state_dict(), "faster_rcnn_buddha_model.pth")

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 151MB/s] 


In [19]:
model.train()
num_epochs = 10
for epoch in range(num_epochs):
    for images, targets in data_loader:
        # Convert images and targets to lists for the batch
        images = list(image for image in images)
        targets = [{k: v for k, v in t.items()} for t in targets]

        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()

    lr_scheduler.step()
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {losses.item()}")

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
 torch.save(model.state_dict(), "faster_rcnn_buddha_model.pth")